Watch DM progress

In [5]:
import os
import time
import numpy as np
import xarray as xr
result_dir = '/usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/'
path = os.path.join(result_dir, 'samples')
in_ds = xr.load_dataset(path + '/uq_modal_beam.nc')
# out_ds = xr.load_dataset(path+'uq_modal_beam_out.nc')

In [6]:
for i in range((315+463+1065+2473+433+704+137+3341)*250,7599653):
    if not i%250:
        print('.', end='', flush=True)        
    jid = in_ds.ids[i].item()
    res_dir = os.path.join(path,jid,f'{jid}_mechanical.npz')
    arr = np.load(res_dir, allow_pickle=True)
    assert arr['self.struct_parms'].item()['dy_tmd'] == in_ds['dD'][i].item()

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

KeyboardInterrupt: 

In [15]:
jid = in_ds.ids[i].item()
res_dir = os.path.join(path,jid,f'{jid}_mechanical.npz')
print(time.ctime(os.stat(res_dir).st_mtime))
print(time.ctime(os.stat(res_dir).st_ctime))

In [2]:
(315+463+1065+2473+433+704+137+3341)*250/729

3062.7572016460904